In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
comp_test=pd.read_csv('/comp_rtest.csv')
comp_test.head()

,Unnamed: 0.1,Unnamed: 0,ID,Tweet,Label,Type
0,2598,1764,NaN,Balurghat is suffering from unnecessary late o...,1,punctuality
1,1033,1839,3885.0,My parents travelled by #garibrath train from ...,Complaint,Cleanliness
2,2215,1069,NaN,#IndianRailways no response from IRCTC,1,facility
3,1124,2012,4058.0,Stopped travelling in #IndianRailways way back...,Complaint,Cleanliness
4,1943,604,NaN,#IndianRailways 02563 BJU NDLS Running late by...,1,punctuality


In [442]:
comp_test['Tweet'][0]


'Balurghat is suffering from unnecessary late of the following  train (13161) continuously. #IndianRailways is incessantly ignoring the sufferings. How does a train take 7 hours for 350 kms &amp; almost  5 hours for rest 100 kms despite auto signal and crossing? Shame #ashwinivaishnaw https://t.co/kG03fPuO3y'

In [ ]:
df_comp= pd.read_csv('/comp_rtrain.csv')
df_comp.head()


,Unnamed: 0.1,Unnamed: 0,ID,Tweet,Label,Type
0,2455,1476,NaN,This is very unfortunate that #IndianRailways ...,1,punctuality
1,2545,1644,NaN,@RailMinIndia @RailwaySeva Kaifiyat Exp 12226 ...,1,punctuality
2,350,612,1158.0,#AshwaniVaishnav #IndianRailways \nHello sir w...,Complaint,Punctuality
3,2397,1370,NaN,@AshwiniVaishnaw \nCould you please help in kn...,1,punctuality
4,2305,1218,NaN,Too much delay in train timing due to derailme...,1,punctuality


In [349]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", tweet)
    tweet = tweet.lower()
    tweet = re.sub(r"\W"," ",tweet)
    tweet = re.sub(r"\d"," ",tweet)
    tweet = re.sub(r"\s+[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+[a-z]$"," ",tweet)
    tweet = re.sub(r"^[a-z]\s+"," ",tweet)
    tweet = re.sub(r"\s+"," ",tweet)
    
    # Remove extra whitespace
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    tweet = re.sub(r'#', '', tweet)

  
    tweet = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
    tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", tweet)
    
    # remove mentions
    tweet = re.sub(r'@', '', tweet)

    
    # remove line breaks
    tweet = re.sub(r'\n', '', tweet)
    return (tweet)

In [ ]:
textbox = list(x)
l=[]
for txt in range(textbox):
    l.append(preprocess_tweet(txt))

In [351]:
comp_test['Tweet'] = comp_test['Tweet'].apply(lambda x: preprocess_tweet(x))


In [352]:
df_comp['Tweet'] = df_comp['Tweet'].apply(lambda x: preprocess_tweet(x))

In [353]:
noice['Tweet'] = noice['tweet'].apply(lambda x: preprocess_tweet(x))

In [354]:
comp_test['Tweet'][0]

'balurghat is suffering from unnecessary late of the following train continuously indianrailways is incessantly ignoring the sufferings how does train take hours for kms amp almost hours for rest kms despite auto signal and crossing shame ashwinivaishnaw'

In [ ]:
df_ncomp= pd.read_csv('/ncomp_rtrain.csv')
df_ncomp.head()

,Unnamed: 0.1,Unnamed: 0,ID,Tweet,Label,Type
0,244,572,1118.0,List of Railway Stations from Rajasthan under ...,Non-Complaint,Railway stations under redevelopment
1,576,1825,3871.0,Did you know that #IndianRailways has the larg...,Non-Complaint,General
2,937,1798,3844.0,Bhopal-Bina MEMU to return on tracks again fr...,Non-complaint,Information
3,1229,99,99.0,🚨 #IndianRailways generated a Record-High Reve...,0,NaN
4,1063,2261,4307.0,Indian Railways’ Shri Ramayana Yatra Train Fro...,Non-complaint,Informational


In [ ]:
ncom_test=pd.read_csv('/ncomp_rtest.csv')
ncom_test.head()

,Unnamed: 0.1,Unnamed: 0,ID,Tweet,Label,Type
0,998,2073,4119.0,#IndianRailways increases the limit of online ...,Non-complaint,Information
1,1621,1137,NaN,Dear @RailMinIndia @IRCTCofficial #IndianRailw...,0,safety
2,611,2014,4060.0,Scenes from the higher standards #durontoexpre...,Non-Complaint,Informational
3,514,1534,3580.0,A train ticket examiners (TTE) checking the ti...,Non-Complaint,News/Information
4,413,1034,3080.0,▶️ https://t.co/snz7nf372c\n\n@tntourismoffcl ...,Non-Complaint,NaN


In [357]:
df_comp['Label'].value_counts()

Complaint    1221
1             830
complaint      14
Name: Label, dtype: int64

In [358]:
comp_test['Label'].value_counts()

Complaint    323
1            225
complaint      2
Name: Label, dtype: int64

In [359]:
df_ncomp['Label'].value_counts()

0                579
Non-Complaint    564
Non-complaint    373
non-complaint      4
Name: Label, dtype: int64

In [360]:
ncom_test['Label'].value_counts()

0                160
Non-Complaint    156
Non-complaint     82
non-complaint      2
Name: Label, dtype: int64

In [361]:
df_comp.shape

(2065, 6)

In [362]:
df_ncomp.shape

(1520, 6)

In [363]:
df_comp['class1']=df_comp['Label'].map({'Complaint': 1,
                                        'complaint': 1,
                                         '1':1})

In [364]:
noice['class1']=noice['label']


In [365]:
comp_test['class1']=comp_test['Label'].map({'Complaint': 1,
                                        'complaint': 1,
                                         '1':1})

In [366]:
comp_test['class1'].value_counts()

1    550
Name: class1, dtype: int64

In [367]:
df_comp['class1'].value_counts()

1    2065
Name: class1, dtype: int64

In [368]:
df_ncomp['class1']=df_ncomp['Label'].map({'Non-Complaint': 0,
                                        'Non-complaint': 0,
                                         'non-complaint':0,
                                         '0':0})

In [369]:
ncom_test['class1']=ncom_test['Label'].map({'Non-Complaint': 0,
                                        'Non-complaint': 0,
                                         'non-complaint':0,
                                         '0':0})

In [370]:
ncom_test['class1'].value_counts()

0    400
Name: class1, dtype: int64

In [371]:
df_ncomp['class1'].value_counts()

0    1520
Name: class1, dtype: int64

In [372]:
df_comp['class1'] = df_comp['class1'].astype(int)
df_ncomp['class1'] = df_ncomp['class1'].astype(int)

In [373]:
noice['class1']=noice['class1'].astype(int)

In [375]:
df_balanced = pd.concat([df_comp, df_ncomp])
df_balanced.shape

(3585, 7)

In [376]:
df_balanced = df_balanced.sample(frac=1, random_state=42)

In [377]:
df_balanced['class1'].value_counts()

1    2065
0    1520
Name: class1, dtype: int64

In [378]:
test_bal=pd.concat([ncom_test, comp_test])
test_bal.shape

(950, 7)

In [379]:
with pd.option_context('display.max_rows', None):
    print(df_comp['Type'].value_counts())



Punctuality                                                   405
punctuality                                                   227
Service                                                       143
Cleanliness                                                   131
Miscellaneous                                                 123
facility                                                      101
Security                                                       69
Crowding                                                       64
Ticket booking                                                 58
cleanliness                                                    53
others                                                         52
Bed Roll                                                       51
Electrical Equioment                                           48
Corruption / Bribery                                           45
Catering & Vending Services                                    41
Political 

In [ ]:
df_comp['class2'] = df_comp['Type'].map({'Punctuality': 0,
                               'punctuality':0,
                                         'Cancelled Train':0,
                                         'Train delay':0,
                                         'Train Delay':0,
                                         'Delay':0,
                                         'Service': 1,
                                         'facility':1,
                                         'Ticket booking':1,
                                         'Refund': 1,
                                         'Services': 1,
                                         'Facilities': 1,
                                         'Difficulty in booking tickets': 1,
                                         'Logistics': 1,
                                         'Fare Charges': 1,
                                         'Ticket/ Seat Availability':1,
                                         'Service feedback':1,
                                         'Service quality': 1,
                                         'Criticism':1,
                                         'Ticket Booking Charge':1,
                                         'Request/Suggestion':1,
                                         'Service issue':1,
                                         'service':1,
                                         'Ticket Booking': 1,
                                         'Miscellaneous': 1,
                                         'miscellaneous': 1,
                                         'Misleaneus':1,
                                         'Medical Assistance':1,
                                         'medical assistance':1,
                                         
                                         'Cleanliness': 2,
                                         'cleanliness': 2,
                                         'Water Availability':2,
                                         'water availability':2,
                                         
                                         
                                         'Political': 3,
                                         'others': 3,
                                         'Others': 3,
                                         
                                            
                                         'Security': 4,
                                         'security': 4,
                                         'loss/found': 4,
                                         'safety': 4,
                                         'Petrol theft during vehicle parceling':4,
                                            
                                            
                                            
                                         'Bed Roll': 5,
                                         'bed roll':5,
                                         'Bed':5,
                                         'Bed roll':5,
                                         
                                         
                                         'Catering & Vending Services': 6,
                                         'food':6,
                                         'catering & vending services':6,
                                         'Food Quality':6,
                                         
                                         
                                            'Electrical Equioment': 7,
                                            'electrical equioment':7,
                                         'Electrical Equipment':7,
                                            'Maintenance': 7,
                                         'maintenance':7,
                                         'Poor condition':7,
                                         
                                         
                                            'Crowding': 8,
                                           'Corruption / Bribery': 8,
                                         'corruption / bribery':8,
                                         
                                         
                                            'Staff Behaviour':9,
                                            'staff behaviour':9,
                                            'Staff Behavior':9,
                               'Staff behavior':9,
                                         'Employee':9,                                
                              })

# drop unused column
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065 entries, 0 to 2064
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  2065 non-null   int64  
 1   Unnamed: 0    2065 non-null   int64  
 2   ID            1362 non-null   float64
 3   Tweet         2065 non-null   object 
 4   Label         2065 non-null   object 
 5   Type          2064 non-null   object 
 6   class1        2065 non-null   int64  
 7   class2        2057 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 129.2+ KB


In [ ]:
comp_test['class2'] = comp_test['Type'].map({'Punctuality': 0,
                               'punctuality':0,
                                         'Cancelled Train':0,
                                         'Train delay':0,
                                         'Train Delay':0,
                                         'Delay':0,
                                         'Service': 1,
                                         'facility':1,
                                         'Ticket booking':1,
                                         'Refund': 1,
                                         'Services': 1,
                                         'Facilities': 1,
                                         'Difficulty in booking tickets': 1,
                                         'Logistics': 1,
                                         'Fare Charges': 1,
                                         'Ticket/ Seat Availability':1,
                                         'Service feedback':1,
                                         'Service quality': 1,
                                         'Criticism':1,
                                         'Ticket Booking Charge':1,
                                         'Request/Suggestion':1,
                                         'Service issue':1,
                                         'service':1,
                                         'Ticket Booking': 1,
                                         'Miscellaneous': 1,
                                         'miscellaneous': 1,
                                         'Misleaneus':1,
                                         'Medical Assistance':1,
                                         'medical assistance':1,
                                         
                                         'Cleanliness': 2,
                                         'cleanliness': 2,
                                         'Water Availability':2,
                                         'water availability':2,
                                         
                                         
                                         'Political': 3,
                                         'others': 3,
                                         'Others': 3,
                                         
                                            
                                         'Security': 4,
                                         'security': 4,
                                         'loss/found': 4,
                                         'safety': 4,
                                         'Petrol theft during vehicle parceling':4,
                                            
                                            
                                            
                                         'Bed Roll': 5,
                                         'bed roll':5,
                                         'Bed':5,
                                         'Bed roll':5,
                                         
                                         
                                         'Catering & Vending Services': 6,
                                         'food':6,
                                         'catering & vending services':6,
                                         'Food Quality':6,
                                         
                                         
                                            'Electrical Equioment': 7,
                                            'electrical equioment':7,
                                         'Electrical Equipment':7,
                                            'Maintenance': 7,
                                         'maintenance':7,
                                         'Poor condition':7,
                                         
                                         
                                            'Crowding': 8,
                                           'Corruption / Bribery': 8,
                                         'corruption / bribery':8,
                                         
                                         
                                            'Staff Behaviour':9,
                                          'staff behaviour':9,
                                          'Staff Behavior':9,
                                          'Staff behavior':9,
                                          'Employee':9,               
                              })

# drop unused column

comp_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  550 non-null    int64  
 1   Unnamed: 0    550 non-null    int64  
 2   ID            358 non-null    float64
 3   Tweet         550 non-null    object 
 4   Label         550 non-null    object 
 5   Type          550 non-null    object 
 6   class1        550 non-null    int64  
 7   class2        550 non-null    int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 34.5+ KB


In [382]:
noice['class2'].value_counts()

1    3000
2    2922
4    1500
0    1500
8    1500
7    1500
9     478
Name: class2, dtype: int64

In [383]:
df_comp['class2'].value_counts()

0.0    636
1.0    501
2.0    217
8.0    143
7.0    129
4.0    127
3.0     94
5.0     81
9.0     68
6.0     61
Name: class2, dtype: int64

In [384]:
df_comp = df_comp.fillna(8)

In [385]:
comp_test=comp_test.fillna(1)

In [386]:
df_comp['class2'] = df_comp['class2'].astype(int)
     

df_comp['class2'].value_counts()


0    636
1    501
2    217
8    151
7    129
4    127
3     94
5     81
9     68
6     61
Name: class2, dtype: int64

In [388]:
comp_test['class2'] = comp_test['class2'].astype(int)
     

comp_test['class2'].value_counts()


1    152
0    151
2     53
8     43
7     31
4     29
6     27
3     23
5     22
9     19
Name: class2, dtype: int64

In [389]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [390]:
token = tokenizer.encode_plus(
    df_comp['Tweet'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [391]:
bal_input_ids=np.zeros((len(df_balanced), 256))
bal_attn_masks = np.zeros((len(df_balanced), 256))

X_input_ids = np.zeros((len(df_comp), 256))
X_attn_masks = np.zeros((len(df_comp), 256))

In [392]:
test_bal_input_ids=np.zeros((len(test_bal), 256))
test_bal_attn_masks = np.zeros((len(test_bal), 256))

test_X_input_ids = np.zeros((len(comp_test), 256))
test_X_attn_masks = np.zeros((len(comp_test), 256))

In [393]:
def generate_training_data(train_df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(train_df['Tweet'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [394]:
X_input_ids, X_attn_masks = generate_training_data(df_comp, X_input_ids, X_attn_masks, tokenizer)
bal_input_ids,bal_attn_masks=generate_training_data(df_balanced, bal_input_ids, bal_attn_masks, tokenizer)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [395]:
test_X_input_ids, test_X_attn_masks = generate_training_data(comp_test, test_X_input_ids, test_X_attn_masks, tokenizer)
test_bal_input_ids ,test_bal_attn_masks=generate_training_data(test_bal, test_bal_input_ids, test_bal_attn_masks, tokenizer)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [396]:
print(X_input_ids.shape)
X_attn_masks.shape

(14465, 256)


(14465, 256)

In [397]:
labels = np.zeros((len(df_comp), 10))
labels.shape
     

(14465, 10)

In [398]:
test_labels = np.zeros((len(comp_test), 10))
test_labels.shape

(550, 10)

In [399]:
labels[np.arange(len(df_comp)), df_comp['class2'].values] = 1 # one-hot encoded target tensor

In [400]:
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [401]:
test_labels[np.arange(len(comp_test)), comp_test['class2'].values] = 1 

In [402]:
test_labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [403]:
lab=df_balanced['class1']

In [404]:
lab.shape

(3585,)

In [405]:
test_lab=test_bal['class1']

In [406]:
test_lab.shape

(950,)

In [407]:
bal_dataset=tf.data.Dataset.from_tensor_slices((bal_input_ids, bal_attn_masks, lab))
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1)
bal_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
def DatasetBalMapFunction(bal_input_ids, bal_attn_masks, lab):
    return {
        'input_ids': bal_input_ids,
        'attention_mask': bal_attn_masks
    },lab
     

# dataset = dataset.map(DatasetMapFunction) # converting to required format for tensorflow dataset 
# dataset.take(1)
bal_dataset=bal_dataset.map(DatasetBalMapFunction)

In [409]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels
     

dataset = dataset.map(DatasetMapFunction) # converting to required format for tensorflow dataset 
     

dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(10,), dtype=tf.float64, name=None))>

In [410]:
bal_dataset = bal_dataset.shuffle(100).batch(16, drop_remainder=True) # batch size, drop any left out tensor
     

bal_dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16,), dtype=tf.int64, name=None))>

In [411]:

dataset = dataset.shuffle(100).batch(16, drop_remainder=True) # batch size, drop any left out tensor
     

dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 10), dtype=tf.float64, name=None))>

In [412]:
len(bal_dataset)

224

In [413]:
len(dataset)

904

In [414]:
p = 0.8
bal_train_size = int((len(df_balanced)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.
     
print(len(df_balanced)//16)
bal_train_size

224


179

In [415]:
bal_train_dataset = bal_dataset.take(bal_train_size)
bal_val_dataset = bal_dataset.skip(bal_train_size)

In [416]:
p = 0.8
train_size = int((len(df_comp)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.
     
print(len(df_comp)//16)
train_size
     

904


723

In [417]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)
     

In [418]:
print(train_dataset)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 10), dtype=tf.float64, name=None))>


In [419]:
print(val_dataset)

<_SkipDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 10), dtype=tf.float64, name=None))>


In [420]:
from transformers import TFBertModel
     

model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [421]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer3')(intermediate_layer)

output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes



In [422]:
binary_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
binary_model.layers [2].trainable = False
binary_model.summary()


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

In [423]:
Boptim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
Bloss_func = tf.keras.losses.BinaryCrossentropy()
Bacc = tf.keras.metrics.BinaryAccuracy('accuracy')


In [424]:
binary_model.compile(optimizer=Boptim,
              loss=Bloss_func,
              metrics=Bacc)

In [425]:
hist = binary_model.fit(
    bal_train_dataset,
    validation_data=bal_val_dataset,
    epochs=10
)
    

Epoch 1/10
179/179 [==============================] - 123s 633ms/step - loss: 0.6040 - accuracy: 0.7032 - val_loss: 0.5575 - val_accuracy: 0.6403
Epoch 2/10
179/179 [==============================] - 87s 485ms/step - loss: 0.4817 - accuracy: 0.8513 - val_loss: 0.4335 - val_accuracy: 0.8764
Epoch 3/10
179/179 [==============================] - 84s 470ms/step - loss: 0.3600 - accuracy: 0.9155 - val_loss: 0.3157 - val_accuracy: 0.9625
Epoch 4/10
179/179 [==============================] - 84s 470ms/step - loss: 0.2765 - accuracy: 0.9284 - val_loss: 0.2406 - val_accuracy: 0.9639
Epoch 5/10
179/179 [==============================] - 83s 464ms/step - loss: 0.2229 - accuracy: 0.9424 - val_loss: 0.1928 - val_accuracy: 0.9625
Epoch 6/10
179/179 [==============================] - 84s 468ms/step - loss: 0.1885 - accuracy: 0.9466 - val_loss: 0.1597 - val_accuracy: 0.9708
Epoch 7/10
179/179 [==============================] - 86s 478ms/step - loss: 0.1693 - accuracy: 0.9448 - val_loss: 0.1368 - val_a

In [ ]:
binary_model.save('binary_model')

INFO:tensorflow:Assets written to: binary_model_97%may/assets


INFO:tensorflow:Assets written to: binary_model_97%may/assets


In [427]:
binary_model.evaluate([test_bal_input_ids ,test_bal_attn_masks],test_lab)

30/30 [==============================] - 50s 1s/step - loss: 0.0965 - accuracy: 0.9789


[0.09645009785890579, 0.9789473414421082]

In [426]:
y= binary_model.predict([test_bal_input_ids ,test_bal_attn_masks])

30/30 [==============================] - 44s 1s/step


In [429]:
new_y = []    # create an empty list to store the new values

for val in y:    # loop over each value in the original list
    if val > 0.5:
        new_y.append(1)    # if the value is greater than 0.5, append 1 to the new list
    else:
        new_y.append(0)

In [430]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(test_lab, new_y))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       400
           1       0.98      0.98      0.98       550

    accuracy                           0.98       950
   macro avg       0.98      0.98      0.98       950
weighted avg       0.98      0.98      0.98       950



In [431]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(2048, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer2')(intermediate_layer)
intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer3')(intermediate_layer)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer4')(intermediate_layer)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer5')(intermediate_layer)

output_layer = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

In [432]:
model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
model.layers [2].trainable = False

model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

In [433]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [434]:
model.compile(optimizer=optim, loss=loss_func, metrics=[acc])
     

hist = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)
     



Epoch 1/10
723/723 [==============================] - 376s 503ms/step - loss: 1.9102 - accuracy: 0.2541 - val_loss: 1.8880 - val_accuracy: 0.2617
Epoch 2/10
723/723 [==============================] - 345s 477ms/step - loss: 1.8586 - accuracy: 0.2915 - val_loss: 1.8387 - val_accuracy: 0.3122
Epoch 3/10
723/723 [==============================] - 333s 461ms/step - loss: 1.8071 - accuracy: 0.3281 - val_loss: 1.7839 - val_accuracy: 0.3284
Epoch 4/10
723/723 [==============================] - 343s 474ms/step - loss: 1.7633 - accuracy: 0.3464 - val_loss: 1.7423 - val_accuracy: 0.3301
Epoch 5/10
723/723 [==============================] - 346s 479ms/step - loss: 1.7118 - accuracy: 0.3760 - val_loss: 1.7025 - val_accuracy: 0.3460
Epoch 6/10
723/723 [==============================] - 347s 479ms/step - loss: 1.6737 - accuracy: 0.3945 - val_loss: 1.6672 - val_accuracy: 0.4012
Epoch 7/10
723/723 [==============================] - 347s 480ms/step - loss: 1.6427 - accuracy: 0.4072 - val_loss: 1.6503 -

In [ ]:
model.save('multi_class_model')

INFO:tensorflow:Assets written to: multi_class_model_2may/assets


INFO:tensorflow:Assets written to: multi_class_model_2may/assets


In [56]:
model = tf.keras.models.load_model('multi_class_model_30april')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Punctuality','Service','Cleanliness','Miscellaneous','Security','Bed Roll','Catering & Vending Services','Maintenance','Refund','Staff Behaviour']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [274]:
input_text = input('Enter movie review here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter movie review here: input_text = input('Enter movie review here: ') processed_data = prepare_data(input_text, tokenizer) result = make_prediction(model, processed_data=processed_data) print(f"Predicted Sentiment: {result}")


1/1 [==============================] - 7s 7s/step
Predicted Sentiment: Service
